In [1]:
import os
import json
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from unsloth import FastVisionModel
from transformers import AutoProcessor, TextStreamer
import nltk

nltk.download("punkt")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Standard import failed for UnslothDDPOTrainer: No module named 'UnslothDDPOTrainer'. Using tempfile instead!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def create_image_caption_dataset(
    image_folder: str,
    captions_json: str,
    caption_strategy: str = 'first'
) -> pd.DataFrame:
    with open(captions_json, 'r') as f:
        captions_data = json.load(f)

    data = []
    for filename, caption_list in captions_data.items():
        image_path = os.path.join(image_folder, filename)
        if not os.path.exists(image_path):
            continue
        try:
            image = Image.open(image_path).convert("RGB")
            caption = caption_list if caption_strategy == 'first' else random.choice(caption_list)
            data.append({"image": image, "caption": caption, "filename": filename})
        except Exception as e:
            print(f"[ERROR] Could not load {filename}: {e}")
    return pd.DataFrame(data)

In [3]:
model_id = "unsloth/Pixtral-12B-2409"  # change if needed

model, tokenizer = FastVisionModel.from_pretrained(
    model_id,
    load_in_4bit=True,  # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for long context
)

model = FastVisionModel.for_inference(model)
processor = AutoProcessor.from_pretrained(model_id)

==((====))==  Unsloth 2025.4.7: Fast Llava patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Llava does not support SDPA - switching to eager!


model.safetensors.index.json:   0%|          | 0.00/214k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

chat_template.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [4]:
image_folder = "/workspace/data/test_dataset"
captions_json = "/workspace/data/test_set.json"

df = create_image_caption_dataset(image_folder, captions_json)
print(f"Loaded {len(df)} image-caption pairs")

Loaded 50 image-caption pairs


In [5]:
print("📌 First 5 entries:")
print(df.head(), "\n")

print("📋 Column types:")
print(df.dtypes)

print("\n🔎 Size")
print(df.shape)

📌 First 5 entries:
                                               image  \
0  <PIL.Image.Image image mode=RGB size=1000x667 ...   
1  <PIL.Image.Image image mode=RGB size=1000x667 ...   
2  <PIL.Image.Image image mode=RGB size=1000x667 ...   
3  <PIL.Image.Image image mode=RGB size=1000x750 ...   
4  <PIL.Image.Image image mode=RGB size=1000x667 ...   

                                             caption    filename  
0  The car exhibits visible damage including a sc...  000481.jpg  
1  The car exhibits significant damage to the win...  000433.jpg  
2  The car exhibits a flat tire on the rear wheel...  000749.jpg  
3  The car exhibits significant damage. The rear ...  000541.jpg  
4  The car shows scratches along the side panel a...  000424.jpg   

📋 Column types:
image       object
caption     object
filename    object
dtype: object

🔎 Size
(50, 3)


In [91]:
import torch
import time
from transformers import TextStreamer
import pandas as pd

streamer = TextStreamer(tokenizer, skip_prompt=True)

def run_vlm_inference(prompt: str, filename: str, df: pd.DataFrame):
    """
    Perform inference on a given image (by filename) from the dataframe using a custom prompt.

    Args:
        prompt (str): The prompt text (can include mask tokens)
        filename (str): Filename of the image in the DataFrame
        df (pd.DataFrame): DataFrame with 'filename' and 'image' columns

    Returns:
        Tuple[str, float, float]: (Generated output, inference time in seconds, VRAM used in GB)
    """
    row = df[df["filename"] == filename]
    if row.empty:
        print(f"[ERROR] No image found with filename: {filename}")
        return None, 0.0, 0.0

    row = row.iloc[0]
    image = row["image"]
    caption = row["caption"]

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image", "image": image}
            ]
        }
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(images=image, text=input_text, return_tensors="pt").to("cuda")

    torch.cuda.reset_peak_memory_stats()
    start_mem = torch.cuda.memory_allocated() / 1024**3  # in GB
    start_time = time.time()

    print(f"🔹 Image: {filename}")
    print(f"🧾 Prompt: {prompt}")
    print("📤 Output:")

    inputs.pop("token_type_ids", None)

    output_ids = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=64,
        use_cache=True,
        temperature=1.5,
        min_p=0.1,
    )

    end_time = time.time()
    end_mem = torch.cuda.max_memory_allocated() / 1024**3

    time_taken = round(end_time - start_time, 3)
    vram_used = round(end_mem - start_mem, 3)
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"⏱️ Time taken: {time_taken} sec | 🧠 VRAM used: {vram_used} GB")
    print("-" * 80)

    return decoded_output, time_taken, vram_used

### Test Images

000404.jpg
000422.jpg
000433.jpg
000481.jpg
000520.jpg
000541.jpg
000698.jpg
000740.jpg
000869.jpg
000889.jpg

In [112]:
filename="000889.jpg"

row = df[df["filename"] == filename]
ground_truth = row.iloc[0]["caption"]

inference_outputs = []
vram_usages = []
inference_times = []

prompts = [
    "",  # Prompt 1: No Prompt
    "Describe &&damage 12 sedan drive’ this !!image.",  # Noisy
    "An image of ....",  # Hand-crafted
    "You are an insurance claims assessor. Provide a detailed description of the car’s condition.",  # Roleplay
    "This <part_1> of the car has <damage_type_1>. The severity appears to be <severity_1>. Additional notes: <text_1>.",  # Masked
    "Describe using format - Damage Type: ___; Affected Part: ___; Severity: ___; Notes: ___"  # Format-Guided
]

for prompt in prompts:
    output, time_taken, vram = run_vlm_inference(prompt, filename, df=df)
    inference_outputs.append(output)
    vram_usages.append(vram)
    inference_times.append(time_taken)

🔹 Image: 000889.jpg
🧾 Prompt: 
📤 Output:
The imageDYjure to the=Gor later. efekt?]
image 1, a Uy of a close to the in schn: a li> p] to open the of the a the xtra  is a the  bef  to been the  s strongNXDoloud
⏱️ Time taken: 4.277 sec | 🧠 VRAM used: 0.601 GB
--------------------------------------------------------------------------------
🔹 Image: 000889.jpg
🧾 Prompt: Describe &&damage 12 sedan drive’ this !!image.
📤 Output:
The command line 1, it'success\s description of the drive`]r a out the 3 S in essence, the 1esa 3s the the$ a the$ to 1l 3w it 0 is i?</s>
⏱️ Time taken: 3.731 sec | 🧠 VRAM used: 0.601 GB
--------------------------------------------------------------------------------
🔹 Image: 000889.jpg
🧾 Prompt: An image of ....
📤 Output:
The impact glass cracking joke, a car  with a few leaded

 ekst-12.0</s>
⏱️ Time taken: 1.92 sec | 🧠 VRAM used: 0.601 GB
--------------------------------------------------------------------------------
🔹 Image: 000889.jpg
🧾 Prompt: You are an insu

In [8]:
from sentence_transformers import SentenceTransformer, util
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
import pandas as pd

sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
def compute_cosine_similarity(reference_captions, generated_caption):
    try:
        total_score = 0.0
        for caption in reference_captions:
            ref_embed = sbert_model.encode(caption, convert_to_tensor=True)
            gen_embed = sbert_model.encode(generated_caption, convert_to_tensor=True)
            score = util.cos_sim(gen_embed, ref_embed).item()
            total_score += score
        avg_score = total_score / len(reference_captions) if reference_captions else 0.0
        return avg_score
    except Exception as e:
        print(f"Error computing cosine similarity: {e}")
        return 0.0

In [10]:
def compute_cider_spice_scores(reference_caption, generated_caption):
    refs = {0: [reference_caption if reference_caption else ""]}
    hypos = {0: [generated_caption if generated_caption else ""]}

    # print(f"Generated caption: {generated_caption}")
    # print(f"Generated hypos: {hypos}")

    ptb = PTBTokenizer()
    refs_tok = ptb.tokenize({i: [{"caption": c} for c in caps] for i, caps in refs.items()})
    hypos_tok = ptb.tokenize({i: [{"caption": hypos[i][0]}] for i in hypos})

    all_scores = {}

    for scorer, name in [(Cider(), "CIDEr"), (Spice(), "SPICE")]:
        try:
            avg_score, _ = scorer.compute_score(refs_tok, hypos_tok)
            if name == "SPICE":
                # SPICE returns dicts per image
                all_scores[name] = avg_score.get("All", {}).get("f", 0.0) if isinstance(avg_score, dict) else avg_score
            else:
                all_scores[name] = avg_score
        except Exception as e:
            print(f"[ERROR] {name} scoring failed: {e}")
            all_scores[name] = 0.0

    return all_scores

In [11]:
from cider import Cider

PICKLE_PATH = "/workspace/data/cardd-df.p"

def compute_cider2_score(reference_caption, generated_caption):
    """
    Computes CIDEr score using new evaluate_cider logic with precomputed DF pickle.

    Args:
        reference_caption (str or list): Ground truth caption(s)
        generated_caption (str): Model output

    Returns:
        float: CIDEr score (averaged if multiple refs)
    """
    refs = {"0": reference_caption if isinstance(reference_caption, list) else [reference_caption]}
    hypos = [{"image_id": "0", "caption": [generated_caption]}]  # Fix: caption should be a list

    cider = Cider()
    score, _ = cider.compute_score(refs, hypos, PICKLE_PATH)

    return score

In [12]:
def evaluate_all_metrics(reference_caption, generated_caption):
    cider_spice_scores = compute_cider_spice_scores(reference_caption, generated_caption)
    cosine_sim = compute_cosine_similarity([reference_caption], generated_caption)
    cider2 = compute_cider2_score(reference_caption, generated_caption)

    return {
        "cosine_similarity": round(cosine_sim, 4),
        #"CIDEr": round(cider_spice_scores.get("CIDEr", 0.0), 4),
        "SPICE": round(cider_spice_scores.get("SPICE", 0.0), 4),
        "CIDEr": round(cider2, 4)
    }

In [113]:
print(ground_truth)
print(inference_outputs)
print(vram_usages)
print(inference_times)

The car exhibits significant damage to the driver's side window, which is shattered extensively. The glass is fragmented and broken, with large pieces missing, indicating a severe impact. 
['The imageDYjure to the=Gor later. efekt?]\nimage 1, a Uy of a close to the in schn: a li> p] to open the of the a the xtra  is a the  bef  to been the  s strongNXDoloud', "Describe &&damage 12 sedan drive’ this !!image.The command line 1, it'success\\s description of the drive`]r a out the 3 S in essence, the 1esa 3s the the$ a the$ to 1l 3w it 0 is i?", 'An image of ....The impact glass cracking joke, a car  with a few leaded\n\n ekst-12.0', 'You are an insurance claims assessor. Provide a detailed description of the car’s condition.As an insurance claims adj. (Image by-vehicle under, insurance adjusters, ins izeled, aaccom/for theiPhone: Kindle ith (an the the the inus F ith a the in of UDOT find:', 'This <part_1> of the car has <damage_type_1>. The severity appears to be <severity_1>. Additional

In [114]:
all_scores = []

for i, gen in enumerate(inference_outputs):
    print(f"\nPrompt {i+1} Output:\n{gen}\nReference:\n{ground_truth}")
    scores = evaluate_all_metrics(ground_truth, gen)
    all_scores.append(scores)


Prompt 1 Output:
The imageDYjure to the=Gor later. efekt?]
image 1, a Uy of a close to the in schn: a li> p] to open the of the a the xtra  is a the  bef  to been the  s strongNXDoloud
Reference:
The car exhibits significant damage to the driver's side window, which is shattered extensively. The glass is fragmented and broken, with large pieces missing, indicating a severe impact. 


PTBTokenizer tokenized 34 tokens at 1079.48 tokens per second.
PTBTokenizer tokenized 46 tokens at 1439.35 tokens per second.


[ERROR] CIDEr scoring failed: Cider.compute_score() missing 1 required positional argument: 'pfile_path'


Parsing reference captions
Parsing test captions


SPICE evaluation took: 588.5 ms

Prompt 2 Output:
Describe &&damage 12 sedan drive’ this !!image.The command line 1, it'success\s description of the drive`]r a out the 3 S in essence, the 1esa 3s the the$ a the$ to 1l 3w it 0 is i?
Reference:
The car exhibits significant damage to the driver's side window, which is shattered extensively. The glass is fragmented and broken, with large pieces missing, indicating a severe impact. 


...

SPICE evaluation took: 611.3 ms

Prompt 6 Output:
Describe using format - Damage Type: ___; Affected Part: ___; Severity: ___; Notes: ___-# whenever i oranges, Car Glass Repair status

 ekst- Impala Dint- oranges- oranges- ingilizes- ing- ing- ing-2[edit- ing- ing- anga- ing- oranges- edit- oranges- edit- ing- edit- ing-
Reference:
The car exhibits significant damage to the driver's side window, which is shattered extensively. The glass is fragmented and broken, with large pieces missing, indicating a severe impact. 


PTBTokenizer tokenized 34 tokens at 1028.07 tokens per second.
PTBTokenizer tokenized 72 tokens at 2276.34 tokens per second.


[ERROR] CIDEr scoring failed: Cider.compute_score() missing 1 required positional argument: 'pfile_path'


Parsing reference captions
Parsing test captions


SPICE evaluation took: 600.1 ms


In [115]:
score_df = pd.DataFrame(all_scores, index=[f"Prompt {i+1}" for i in range(len(inference_outputs))])
score_df

,cosine_similarity,SPICE,CIDEr
Prompt 1,0.1576,0.0000,3.3421
Prompt 2,0.3584,0.0606,2.2063
Prompt 3,0.5442,0.1538,2.1437
Prompt 4,0.2248,0.0435,1.4011
Prompt 5,0.4149,0.0408,3.0305
Prompt 6,0.4410,0.0000,0.0000


In [18]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.styles import Alignment
from PIL import Image as PILImage
import os

def log_prompt_metrics_to_excel(
    filename: str,
    model_name: str,
    inference_outputs: list,
    metrics: list,
    inference_times: list,
    vram_usages: list,
    df: pd.DataFrame,
    output_excel_path: str = "prompt_tuning_results_cardd_pixtral.xlsx"
):
    row = df[df["filename"] == filename].iloc[0]
    original_caption = row["caption"]
    image_path = os.path.join("/workspace/data/test_dataset", filename)

    # Load or create workbook
    if os.path.exists(output_excel_path):
        wb = load_workbook(output_excel_path)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = "Prompt Evaluation"
        headers = [
            "Model", "Image Number", "Image", "Original Caption",
            "Prompt", "Output", "Inference Time (s)", "VRAM Used (GB)",
            "CIDEr", "SPICE", "Cosine Similarity"
        ]
        ws.append(headers)

    # Find the next empty row
    start_row = ws.max_row + 1

    # Append data
    for i in range(6):
        ws.append([
            model_name,
            filename,
            "",  # placeholder for image
            original_caption,
            f"Prompt {i+1}",
            inference_outputs[i],
            inference_times[i],
            vram_usages[i],
            metrics[i]["CIDEr"],
            metrics[i]["SPICE"],
            metrics[i]["cosine_similarity"]
        ])

    # Merge A–D columns across the 6 rows
    for col in ["A", "B", "C", "D"]:
        ws.merge_cells(f"{col}{start_row}:{col}{start_row + 5}")

    # Apply alignment
    align_top_wrap = Alignment(wrap_text=True, vertical="top")
    align_top = Alignment(vertical="top")

    for row in range(start_row, start_row + 6):
        for col_letter in ["A", "B", "C", "E"]:
            ws[f"{col_letter}{row}"].alignment = align_top
        for col_letter in ["D", "F"]:
            ws[f"{col_letter}{row}"].alignment = align_top_wrap
        ws.row_dimensions[row].height = 120

    # Set column widths
    ws.column_dimensions["D"].width = 40  # Original Caption
    ws.column_dimensions["F"].width = 40  # Output

    # Embed image
    if os.path.exists(image_path):
        print(f"[INFO] Inserting image: {image_path}")
        try:
            img = ExcelImage(image_path)
            img.width = 150
            img.height = 150
            img.anchor = f"C{start_row}"
            ws.add_image(img)
        except Exception as e:
            print(f"[ERROR] Could not insert image: {e}")

    wb.save(output_excel_path)
    print(f"✅ Logged metrics for {filename} to {output_excel_path}")

In [116]:
log_prompt_metrics_to_excel(
    filename=filename,
    model_name=model_id,
    inference_outputs=inference_outputs,
    metrics=all_scores,
    inference_times=inference_times,
    vram_usages=vram_usages,
    df=df
)

[INFO] Inserting image: /workspace/data/test_dataset/000889.jpg
✅ Logged metrics for 000889.jpg to prompt_tuning_results_cardd_pixtral.xlsx
